In [36]:
# Librerie principali
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# PyTorch & torchvision
import torch
from torchvision import models
from torchvision.models import ResNet50_Weights

# OpenCV
import cv2

# Similarità
from sklearn.metrics.pairwise import cosine_distances

Setup

In [33]:
# Se CUDA disponibile, usa la GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carica ResNet50 pre-addestrata e in modalità eval
weights = ResNet50_Weights.IMAGENET1K_V1  # o DEFAULT per i pesi più aggiornati
model = models.resnet50(weights=weights)
model.eval()
model.to(device)

# Preprocessing standard per ResNet
preprocess = weights.transforms()

# Carica features da file .npz
data = np.load("resnetfc.npz", allow_pickle=True)
feature_matrix = data["features"]
filenames = data["filenames"]
labels = data["labels"]



Funzione Estrazione Feature

In [29]:
def extract_fc_features_from_image(image_path, model, preprocess, device):
    """
    Estrae le feature del layer fully connected (fc) di ResNet50 da un'immagine.
    """
    try:
        img = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"[ERRORE] Immagine non valida {image_path}: {e}")
        return None

    img_tensor = preprocess(img).unsqueeze(0).to(device)
    fc_output = []

    def hook_fn(module, input, output):
        fc_output.append(output)

    hook = model.fc.register_forward_hook(hook_fn)
    with torch.no_grad():
        model(img_tensor)
    hook.remove()

    if fc_output:
        return fc_output[0].squeeze(0).cpu().numpy()
    else:
        print(f"[ERRORE] Nessun output FC per {image_path}")
        return None

Elaborazione Batch e Salvataggio Feature in .npz

In [38]:
def process_and_save_features(base_folder, subfolders, output_file):
    """
    Estrae le feature FC da immagini in più cartelle e salva in un file .npz.
    """
    all_features = []
    all_filenames = []
    all_labels = []

    for label in subfolders:
        folder_path = os.path.join(base_folder, label)
        if not os.path.isdir(folder_path):
            print(f"[ATTENZIONE] Cartella non trovata: {folder_path}")
            continue
        print(f"[INFO] Elaboro cartella: {label}")

        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tif')):
                img_path = os.path.join(folder_path, filename)
                features = extract_fc_features_from_image(img_path, model, preprocess, device)
                if features is not None:
                    all_features.append(features)
                    all_filenames.append(filename)
                    all_labels.append(label)
                else:
                    print(f"[ERRORE] Feature non estratte da {img_path}")

    # Salva in file .npz
    np.savez(output_file,
             features=np.array(all_features),
             filenames=np.array(all_filenames),
             labels=np.array(all_labels))
    
    print(f"[SALVATO] Features salvate in {output_file}")
    print(f"[FINE] Totale immagini processate: {len(all_features)}")

 Retrieval: Immagini più Simili (distanza coseno)

In [39]:
def find_k_similar_cosine(img_path, k):
    """
    Trova le k immagini più simili rispetto a una query, usando la distanza coseno.
    """
    query_feature = extract_fc_features_from_image(img_path, model, preprocess, device)
    if query_feature is None:
        print("[ERRORE] Impossibile estrarre feature dalla query.")
        return

    query_feature = np.array(query_feature).reshape(1, -1)
    distances = cosine_distances(feature_matrix, query_feature).flatten()

    top_k_idx = np.argsort(distances)[:k]
    top_k_scores = distances[top_k_idx]

    print(f"\nTop {k} immagini più simili a: {img_path}")
    for rank, idx in enumerate(top_k_idx):
        print(f"{rank+1}. {filenames[idx]} | Classe: {labels[idx]} | Distanza: {top_k_scores[rank]:.4f}")

    # Visualizzazione
    fig, axs = plt.subplots(1, k+1, figsize=(15, 5))
    axs[0].imshow(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
    axs[0].set_title("Query")
    axs[0].axis('off')

    for i, idx in enumerate(top_k_idx):
        match_img = cv2.imread(os.path.join("Part1", labels[idx], filenames[idx]))
        axs[i+1].imshow(cv2.cvtColor(match_img, cv2.COLOR_BGR2RGB))
        axs[i+1].set_title(f"Rank {i+1}\nD={top_k_scores[i]:.4f}")
        axs[i+1].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# Parametri cartelle e output
base_folder = "Part1"
subfolders = ["brain_glioma", "brain_menin", "brain_tumor"]
output_file = "resnetfc.npz"

# Estrazione e salvataggio
process_and_save_features(base_folder, subfolders, output_file)

In [ ]:

# Test su immagine di query
query_img = "Part1/brain_glioma/brain_glioma_0001.jpg"
find_k_similar_cosine(query_img, k=5)

codice non pre-modifiche

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt

def extract_resnet_fc_features(img_path):
    # Carica l'immagine e assicura la modalità RGB
    img = Image.open(img_path).convert('RGB')
    
    # Pre-processamento standard per ResNet: ridimensiona, ritaglia, trasforma in tensor e normalizza
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
    img_tensor = preprocess(img).unsqueeze(0)  # Aggiungi la dimensione del batch
    
    # Carica la ResNet50 pre-addestrata e mettila in modalità eval
    resnet = models.resnet50(pretrained=True)
    resnet.eval()
    
    # Lista per salvare l'output del layer fc
    fc_output = []

    # Definisci un hook che memorizza l'output del layer fc
    def hook_fn(module, input, output):
        fc_output.append(output)
    
    # Registra il forward hook sul layer fully connected
    hook = resnet.fc.register_forward_hook(hook_fn)
    
    # Passa l'immagine attraverso il modello
    with torch.no_grad():
        resnet(img_tensor)
    
    # Rimuovi l'hook (è buona norma pulirlo dopo l'uso)
    hook.remove()
    
    # Il risultato è salvato in fc_output[0] con shape [1, 1000]. Rimuovi la dimensione del batch:
    features = fc_output[0].squeeze(0)
    
    # Stampa e visualizza il vettore delle features
    print("Vettore delle features (1000 elementi):")
    print(features.numpy())
    
    # Visualizzazione opzionale: ad esempio, un bar plot dei primi 20 valori
    plt.figure(figsize=(10, 4))
    plt.bar(range(20), features.numpy()[:20])
    plt.xlabel("Indice feature")
    plt.ylabel("Valore")
    plt.title("Prime 20 features del layer fc")
    plt.show()
    
    return features

# Esempio di esecuzione
img_path = "Part1/brain_glioma/brain_glioma_0002.jpg"  # Sostituisci con il percorso corretto della tua immagine
features = extract_resnet_fc_features(img_path)


Task: 2

In [ ]:
import os
import csv
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

def extract_fc_features_from_image(image_path, model, preprocess, device):
    """
    Apre l'immagine, la pre-processa e la passa attraverso la rete.
    Utilizza un hook per catturare l'output del layer fully connected (fc) della ResNet.
    Restituisce un vettore numpy di dimensione 1000.
    """
    try:
        img = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Errore nell'apertura dell'immagine {image_path}: {e}")
        return None

    img_tensor = preprocess(img).unsqueeze(0).to(device)
    
    fc_output = []

    # Definizione del hook per catturare l'output del layer fc
    def hook_fn(module, input, output):
        fc_output.append(output)

    hook = model.fc.register_forward_hook(hook_fn)
    with torch.no_grad():
        model(img_tensor)
    hook.remove()

    # fc_output[0] ha forma [1, 1000], rimuoviamo la dimensione batch e riportiamo su CPU
    features = fc_output[0].squeeze(0).cpu().numpy()
    return features

def main():
    # Imposta il percorso della cartella base e le sottocartelle
    base_dir = "Part1"
    subfolders = ["brain_glioma", "brain_menin", "brain_tumor"]
    image_extensions = (".jpg", ".jpeg", ".png", ".bmp")

    image_paths = []
    # Cerca le immagini all'interno delle sottocartelle specificate
    for sub in subfolders:
        dir_path = os.path.join(base_dir, sub)
        if os.path.isdir(dir_path):
            for root, _, files in os.walk(dir_path):
                for file in files:
                    if file.lower().endswith(image_extensions):
                        image_paths.append(os.path.join(root, file))
        else:
            print(f"La cartella {dir_path} non esiste.")

    if not image_paths:
        print("Non sono state trovate immagini nelle cartelle specificate.")
        return

    # Configura il dispositivo da utilizzare (GPU se disponibile, altrimenti CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Definisci la trasformazione di pre-processing standard per ResNet
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

    # Carica il modello ResNet50 pre-addestrato ed impostalo in modalità evaluation
    model = models.resnet50(pretrained=True)
    model.eval()
    model.to(device)


    """""
    # Prepara i dati che verranno salvati sul file CSV
    csv_rows = []
    # Crea l'header: la prima colonna è il percorso dell'immagine, le altre sono le feature
    header = ["image_path"] + [f"fc_feature_{i}" for i in range(1000)]
    csv_rows.append(header)

    # Estrai le features per ogni immagine e conserva il risultato in una riga di CSV
    for img_path in image_paths:
        features = extract_fc_features_from_image(img_path, model, preprocess, device)
        if features is not None:
            row = [img_path] + features.tolist()
            csv_rows.append(row)
        else:
            print(f"Errore nell'estrazione delle features da {img_path}")

    # Salva i risultati in un file CSV
    csv_file = "resnetfc.csv"
    try:
        with open(csv_file, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerows(csv_rows)
        print(f"Salvataggio completato in {csv_file}")
    except Exception as e:
        print(f"Errore durante il salvataggio del file CSV: {e}")

if __name__ == "__main__":
    main()
"""

# === Funzione per elaborare immagini in più cartelle ===
def process_and_save_features(base_folder, subfolders, output_file):
     # Carica il modello ResNet50 pre-addestrato ed impostalo in modalità evaluation
    model = models.resnet50(pretrained=True)
    model.eval()
    model.to(device)

    all_features = []
    all_filenames = []
    all_labels = []

    for label in subfolders:
        folder_path = os.path.join(base_folder, label)
        print(f"[INFO] Elaboro cartella: {label}")
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tif')):
                img_path = os.path.join(folder_path, filename)
                features = extract_fc_features_from_image(img_path, model, preprocess, device)
                if features is not None:
                    all_features.append(features)
                    all_filenames.append(filename)
                    all_labels.append(label)

    # Converti in array
    features_array = np.array(all_features)
    filenames_array = np.array(all_filenames)
    labels_array = np.array(all_labels)

    # Salva in .npz
    np.savez(output_file, features=features_array, filenames=filenames_array, labels=labels_array)
    print(f"[SALVATO] Features salvate in {output_file}")
    print(f"[FINE] Totale immagini processate: {len(all_features)}")


# === Esecuzione ===
base_folder = "Part1"
subfolders = ["brain_glioma", "brain_menin", "brain_tumor"]
output_file = "resnetfc0.npz"

process_and_save_features(base_folder, subfolders, output_file)

In [ ]:
from torchvision import models
import cv2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model.eval()
model.to(device)

In [ ]:
import os
import csv
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

def extract_fc_features_from_image(image_path, model, preprocess, device):
    """
    Apre l'immagine, la pre-processa e la passa attraverso la rete.
    Utilizza un hook per catturare l'output del layer fully connected (fc) della ResNet.
    Restituisce un vettore numpy di dimensione 1000.
    """
    try:
        img = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"[ERRORE] Immagine non valida {image_path}: {e}")
        return None

    img_tensor = preprocess(img).unsqueeze(0).to(device)
    
    fc_output = []

    def hook_fn(module, input, output):
        fc_output.append(output)

    hook = model.fc.register_forward_hook(hook_fn)
    with torch.no_grad():
        model(img_tensor)
    hook.remove()

    if fc_output:
        return fc_output[0].squeeze(0).cpu().numpy()
    else:
        print(f"[ERRORE] Nessun output FC per {image_path}")
        return None

def process_and_save_features(base_folder, subfolders, output_file):
    # Configura il dispositivo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Definisci la trasformazione
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

    # Carica il modello
    model = models.resnet50(pretrained=True)
    model.eval()
    model.to(device)

    all_features = []
    all_filenames = []
    all_labels = []

    for label in subfolders:
        folder_path = os.path.join(base_folder, label)
        if not os.path.isdir(folder_path):
            print(f"[ATTENZIONE] Cartella non trovata: {folder_path}")
            continue
        print(f"[INFO] Elaboro cartella: {label}")
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tif')):
                img_path = os.path.join(folder_path, filename)
                features = extract_fc_features_from_image(img_path, model, preprocess, device)
                if features is not None:
                    all_features.append(features)
                    all_filenames.append(filename)
                    all_labels.append(label)
                else:
                    print(f"[ERRORE] Feature non estratte da {img_path}")

    # Converti in array
    features_array = np.array(all_features)
    filenames_array = np.array(all_filenames)
    labels_array = np.array(all_labels)

    # Salva in .npz
    np.savez(output_file, features=features_array, filenames=filenames_array, labels=labels_array)
    print(f"[SALVATO] Features salvate in {output_file}")
    print(f"[FINE] Totale immagini processate: {len(all_features)}")

# === Esecuzione ===
if __name__ == "__main__":
    base_folder = "Part1"
    subfolders = ["brain_glioma", "brain_menin", "brain_tumor"]
    output_file = "resnetfc.npz"

    process_and_save_features(base_folder, subfolders, output_file)

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

# Carica dati da file .npz (con feature ResNetFC)
data = np.load("resnetfc.npz", allow_pickle=True)
feature_matrix = data["features"]
filenames = data["filenames"]
labels = data["labels"]

# Definisci la trasformazione
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# === Funzione per trovare k immagini più simili (coseno) ===
def find_k_similar_cosine(img_path, k):
    query_feature = extract_fc_features_from_image(img_path, model, preprocess, device)  # Deve restituire un vettore 1D
    if query_feature is None:
        print("[ERRORE] Impossibile estrarre feature dalla query.")
        return

    query_feature = np.array(query_feature).reshape(1, -1)

    # Calcola la distanza del coseno (1 - similarità coseno)
    distances = cosine_distances(feature_matrix, query_feature).flatten()

    top_k_idx = np.argsort(distances)[:k]
    top_k_scores = distances[top_k_idx]

    print(f"\nTop {k} immagini più simili (distanza coseno) a: {img_path}")
    for rank, idx in enumerate(top_k_idx):
        print(f"{rank+1}. {filenames[idx]} | Classe: {labels[idx]} | Distanza: {top_k_scores[rank]:.4f}")

    # Visualizza risultati
    fig, axs = plt.subplots(1, k+1, figsize=(15, 5))
    axs[0].imshow(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
    axs[0].set_title("Query")
    axs[0].axis('off')

    for i, idx in enumerate(top_k_idx):
        match_img = cv2.imread(os.path.join("Part1", labels[idx], filenames[idx]))
        axs[i+1].imshow(cv2.cvtColor(match_img, cv2.COLOR_BGR2RGB))
        axs[i+1].set_title(f"Rank {i+1}\nD={top_k_scores[i]:.4f}")
        axs[i+1].axis('off')

    plt.tight_layout()
    plt.show()

# === Test ===
query_img = "Part1/brain_glioma/brain_glioma_0001.jpg"
find_k_similar_cosine(query_img, k=5)